In [36]:
class Operad:
    def __init__(self):
        self.colors = {}
        self.operations = {}

    def add_color(self, name, prop={}):
        if t:=self.colors.get(name):
            self.colors[name] = (t, prop)
        else:
            self.colors[name] = prop
        return name, t

    def add_operation(self, name, prop={}):
        self.operations[name] = prop
        return name


class Tree:
    def __init__(self, operation, trunk, branches, operad):

        trunk, parent = operad.add_color(trunk, self)
        self.trunk = trunk
        if parent:
            parent.branches[trunk] = self
            self.depth = parent.depth + 1
        else:
            self.depth = 0
        
        self.branches = {
            operad.add_color(branch, self)[0]: uTree(branch, depth=self.depth) for branch in branches
        }
        self.node = operad.add_operation(operation, self)

    def print_edges(self):
        return "".join(
            [f"{str(self.trunk)}\n"]
            + [
                "\t" * (self.depth + 1) + f"{branch.print_edges()}\n"
                for branch in self.branches.values()
            ]
        )

    def print_nodes(self):
        return "".join(
            [f"{str(self.node)}\n"]
            + [
                "\t" * (self.depth + 1) + f"{branch.print_nodes()}\n"
                for branch in self.branches.values()
            ]
        )

    def __str__(self):
        return f"{self.node}({','.join(self.branches.keys())};{self.trunk})"


class uTree(Tree):
    def __init__(self, name, depth):
        self.trunk = name
        self.branches = {}
        self.node = None
        self.depth = depth
    


In [41]:
import re
def string_to_tree_space(string, operad):

    operads = []
    operations = string.split("|")
    regex = r"(.*)\((.*)\)"
    for operation in operations:
        match = re.match(regex, operation)
        if not match:
            raise RuntimeError(f"Operation not defined correctly {operation}")
        operation, parameters = match.groups()
        branches, trunk = parameters.split(";")
        operads.append(Tree(operation, trunk, branches.split(","), operad))
    return operads[0], operad


def tree_space_to_string(tree_space):
    _, operad = tree_space
    return "|".join(str(tree) for tree in operad.operations.values())


# S = string_to_tree_space("o0w(1,2;0)", Operad())
# T = string_to_tree_space("o0b(b,c;a)", Operad())
S = string_to_tree_space("0W(1;0)|1W(2,3;1)", Operad())
T = string_to_tree_space("0B(b,d;a)|1B(c;b)|2B(e;d)", Operad())
print(tree_space_to_string(S))
print(tree_space_to_string(T))

print(S[0].print_nodes())

0W(1;0)|1W(2,3;1)
0B(b,d;a)|1B(c;b)|2B(e;d)
0W
	1W
		None

		None





In [45]:
from copy import deepcopy

class DualTree: 
    def __init__(self, S, T):
        self.S_og_space = deepcopy(S)
        self.T_og_space = deepcopy(T)
        self.dual_tree = deepcopy(S[0])
        self.T = deepcopy(T[0])
        
        self.add_color_base(self.dual_tree, self.T.trunk)
        
    def add_color_edge(self, T, c, depth):
        T.trunk = (c, T.trunk)
        T.depth += depth
        for Ti in T.branches.values():
            self.add_color_edge(Ti, c, depth)
        return T
        
    def add_color_base(self, S, c):
        S.trunk = (S.trunk, c)
            
        for i, Si in S.branches.items():
            if isinstance(Si, uTree):
                S.branches[i] = self.add_color_edge(deepcopy(self.T), i, S.depth+1)
            else:
                self.add_color_base(Si, c)
            

DT = DualTree(S,T)
print(DT.dual_tree.print_nodes())

0W
	1W
		0B
			1B
				None


			2B
				None



		0B
			1B
				None


			2B
				None





